[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/EMBL-EBI-imaging-course-04-2025/blob/main/ReadingData_fromIDR.ipynb)
# Reading binary data from IDR

## Learning Objectives

* How to load the binary using a URL
* How to load the binary using the API

## Reading studies from IDR

To re-analyse studies available in IDR, it is preferable to download the studies from IDR to read the binaries data
Please follow the instruction available [here](https://idr.openmicroscopy.org/about/download.html).
Depending on the image formats, you will need a suitable library to read the data. The Java library [Bio-Formats](https://www.openmicroscopy.org/bio-formats/)
is the reference for reading data. Several Python libraries also exist to read the data, for example [BioIO](https://github.com/bioio-devs/bioio) library has a plugin offering a wrapper around the Bio-Formats.

## Reading an image binary from IDR

It is sometimes useful to only download one image of studies. The [imageio.v3](https://imageio.readthedocs.io/en/stable/reference/core_v3.html) can directly read the data from a given url but it **does not support** all imaging formats.

If the image format is not supported, you can use, for example, ``curl`` or ``wget`` to download the data, then either convert the data or use another library.

To find the path to the image
![path](./includes/web_path_to_image.png)

In [1]:
BASE_RESOURCE = "https://ftp.ebi.ac.uk/pub/databases/IDR/"
path = BASE_RESOURCE + "idr0062-blin-nuclearsegmentation/20190429-ftp/Blastocysts/B4_C3.tif" 

In [ ]:
import imageio.v3 as iio
img = iio.imread(path)
print(img.shape)

In [ ]:
z = 100
c = 1
data = img[z, c, :, :]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(data)

## Reading an image binary from IDR using API
As mentioned previously, downloading a file will then require to have a suitable reader install locally. This might not always be possible or not suitable at all depending on the format. An image might be composed of multiple files on disk so downloading on file could be incorrect.
You can nonetheless download binary data from IDR (and OMERO) using the Python API

In [ ]:
### Install the libraries to connect to idr

# Ice Python binding
%pip install https://github.com/glencoesoftware/zeroc-ice-py-linux-x86_64/releases/download/20240202/zeroc_ice-3.6.5-cp311-cp311-manylinux_2_28_x86_64.whl

# Package required to interact with IDR or OMERO
%pip install omero-py

### Connect to IDR

In [ ]:
from omero.gateway import BlitzGateway
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

In [ ]:
image_id = 6001247

In [ ]:
image = conn.getObject("Image", image_id)

Retrieve information about the image

In [ ]:
pixels = image.getPrimaryPixels()
size_c = image.getSizeC()
size_x = image.getSizeX()
size_y = image.getSizeY()
size_t = image.getSizeT()
size_z = image.getSizeZ()
print(size_c)

In [ ]:
import numpy
z, t = 100, 0 # set a timepoint and a Z-section
c_list = []
for c in range(size_c):  # all channels
    c_list.append((z, c, t))

    values = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(c_list)
    print("Downloading image %s" % image.getName())
    all_planes = numpy.stack(list(planes))
# Reshape to match what we have using the imageio
shape = (size_c, size_y, size_x)
data_reshaped = numpy.reshape(all_planes, newshape=shape)

In [ ]:
print(data_reshaped.shape)

In [ ]:
data_from_idr = data_reshaped[1, :, :,]

In [ ]:
plt.imshow(data_from_idr)

**Exercise:**

Write a ``for`` loop to load all the planes
Check the dimension of the Numpy array
Reshape the area to ``size_t, size_c, size_z, size_y, size_x`` if it is not already available in that odrder

### Load ROI associated to an image
Region of Interest or ROIs usually resulting from analysis are saved independantly of the original binary data
ROIs can also be retrieved using the API. The supported elements are described in the [OME ROI model](https://ome-model.readthedocs.io/en/stable/developers/roi.html)

In [ ]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(image_id, None)

shapes = []
for roi in result.rois:
    shapes.append(roi.copyShapes())

ROIs associated to the image are masks so in order to visualise them, we need to convert them to numpy array.
For that we first need to install to a conversion library

In [ ]:
%pip install omero-cli-zarr

In [ ]:
from omero_zarr import masks
dims = (size_t, size_c, size_z, size_y, size_x)
saver = masks.MaskSaver(None, image, numpy.int64)
labels, fillColors, properties = saver.masks_to_labels(shapes, mask_shape=dims)

Display the labels on top of the image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
c = 1
t = 0
z = 100
img_labels = labels[t, c, z, :, :]
plt.imshow(data_reshaped[c, :, :])
plt.imshow(img_labels, cmap='jet', alpha=0.5*(img_labels>0)) 

### Disconnect from IDR

In [ ]:
conn.close()

### License (BSD 2-Clause)¶

Copyright (C) 2025 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.